<a href="https://colab.research.google.com/github/annikaaross/Homochirality-project/blob/annika-branch/Tidied_Homochirality_v005.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homochirality Model

I have changed this Hello and welcome to *Do Opposites Attract?*, the first monomer dating show. We're here to see if similarity between member(s) of a relationship matters in the strength of the bond between the individuals. Read on as we introduce you to our monomers, our polymers, and the stylish bag that they reside in. Along the way we will keep you up to date on all the drama of falling in love and breaking hearts. 

## Global Parameters

Here's where we keep the rules by which our contestants must play.

In [1]:
#the bond break probbility applied to all bonds
BASE_BOND_BREAK_PROBABILITY = 0.2
#if a bond itself is homochiral the base bond fator is multiplied by
#this factor
HOMOCHIRAL_BREAK_FACTOR = 0.9
#this factor is incorporated for each additional neighboring homochiral bond
# becomes less and less influential as the bond gets further away
HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR = 0.3
#decreases the break probability based on the length of the polymer
LENGTH_FACTOR = 0.6

#limits the influence of length on breakprobabiltiy(the larger the less influence it has)
N=30

#saves the break probabilities of sequences of polymers
break_prob_lookup_table = {}

poisson_dict={}

#used in Poisson distribution
LAMBDA=6
#################################################################
#these are the same factors as before but with on for left and  #
#one for right homochiral bonds.                                #
#used in biaseastbondbreakability method which can be choosen to#
#be used over the regular eastbondbreakability method in the    #
#self.reset_break_probability() method                          #
#################################################################
HOMOCHIRAL_BREAK_FACTOR_LEFT = 0.0
HOMOCHIRAL_BREAK_FACTOR_RIGHT = 0.9
HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT = 0.3
HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT = 0.3


#These are the liklihoods that a monomer will choose to poof or bond, respectively.
#Used in the Reactables class.
POOF_CHANCE = 0.3333
BOND_PROB = 0.3333


## Imports and Helpers

First we install a package called ```jdc``` which allows us to define classes across different cells using the syntax

```
%%add_to our_class
def our_function(self, our_variable):
  print our_variable
```

The documentation on this magic function can be found at https://alexhagen.github.io/jdc/






In [2]:
!pip install jdc

Here are the imports.

In [3]:
import random
import numpy as np
import copy
from google.colab import widgets
import matplotlib.pyplot as plt
import jdc
import more_itertools

And here are some helper functions.

In [4]:
def make_pool(n):
  """
  Return a list of n new monomers.
  """
  monomers = []
  for n in range(n):
    monomers.append(Monomer())
  return monomers


def numpy_fillna(data):
  """ Rectangularize a jagged array.

  Source: https://stackoverflow.com/a/32043366
  """
  # Get lengths of each row of data
  lens = np.array([len(i) for i in data])

  # Mask of valid places in each row
  mask = np.arange(lens.max()) < lens[:,None]

  # Setup output array and put elements from data into masked positions
  out = np.zeros(mask.shape, dtype=float)
  out[mask] = np.concatenate(data)
  return out

## The Monomer class

These are our Monomers. The individuals in our dating show. They are the best thing a girl can be in this worl, beautiful little fools. They know that they exist and they know what kind of monomer they are, left or right(thats handedness not political party alignment) What more can you ask for?

### Constructor

In [5]:
class Monomer:

  def __init__(self, **kwargs):
    """
    The constructor method for Monomer objects. Assigns default handedness.
    Accepts argument hand = bool with keyword.
    Considering other syntax for passing in handedness.
    """
    import random
    self._handedness = kwargs['hand'] if 'hand' in kwargs else random.choice([True,False])
    self._eastbrkprob = kwargs['brkprob'] if 'brkprob' in kwargs else -1
  
  def __repr__(self):
    return str(f"{self.get_handedness()}-monomer")


### Getters and Setters

In [6]:
%%add_to Monomer

def get_handedness(self):
  """Getter method for Monomer handedness."""
  return self._handedness

def get_eastbrkprob(self):
  """Getter method for east bond brk probability"""
  return self._eastbrkprob

def set_eastbrkprob(self, newbrk):
  """Setter method for east bond brk probability"""
  self._eastbrkprob = newbrk

def reset_eastbrkprob(self) :
  """reset east brk probability to -1 
      to be called when single monomer is broken from polymer"""
  self.set_eastbrkprob(-1)


### Polymer Compatibility

In [7]:
%%add_to Monomer

def get_chirality(self):
  """ Return an expression of handedness compatible with the Polymer notation
  of chirality (n_lefts, n_rights).
  """
  if self.get_handedness():
    return (1,0)
  else:
    return (0,1)

def get_length(self):
  """ Return 1. """
  return 1

def get_ee(self):
  """ Return the enantiomeric excess of the monomer. This value is always 1. """
  return 1

def get_signed_ee(self):
  """ Return the signed enantiomeric excess of the monomer.
  Returns 1 if left handed, -1 if right handed.
  """
  if self.get_handedness():
    return 1
  else:
    return -1

def generate_old_log(self):
  """ Return a log tuple in the format
  (length, signed ee, total # homochiral bonds)
  """
  return (self.get_length(), 
          self.get_signed_ee())
          #sum(self.get_leftrighthomochiralbonds()))

def generate_log(self):
  """ Return a log compatible with the reactables history. """
  return (self.get_handedness(),)

## The Polymer class

This is our Polymer class, aka the relationships in this dating show. They contain a list of all the monomers that are inside of them. It is kept in a specific order. They are pretty open books. They can tell us how many monomers they contain, they add new members(either monomers or whole other polymers), they can remove entire groups of members after a certain index, they can check which of their bonds are homochiral and use this to calculate probability of a certain bond breaking(the weak areas in their relationship). They can use this to then tell the producers if and where they are planning on breaking off members of they ranks. Like all good relationships, they use purely mathematics and chance to determine these heart breaking decisions. They can also tell us how many left handed monomers and how many right handed monomers they contain. 

### Constructor

In [8]:
class Polymer:
  def __init__(self, monomers = []):
    """ Constructor method for Polymer.
    In addition to setting the monomers list, also calculates all break probabilities
    of the bonds it contains using reset_break_probability.
    """
    self._monomers = monomers
    #self.reset_break_probability()

  def __str__(self):
    """ str function for Polymer class. """
    return f"\nA polymer of length {self.get_length()}: {self.get_sequence()}"
  
  def __repr__(self):
    """ repr function for Polymer class. """
    return f"{self.get_sequence()}-polymer"

### Getters, Setters, and Logs

In [9]:
%%add_to Polymer


def get_monomers(self):
  """ Return list of monomers in the polymer. """
  return self._monomers


def get_length(self):
  """ Return the number of monomers in the polymer. """
  return len(self._monomers)


def get_sequence(self):
  """ Return the sequence of monomers in the polymer in a human-readable (and loggable) format. """
  sequence = []
  for monomer in self.get_monomers():
    sequence.append(monomer.get_handedness())
  return sequence


def set_monomers(self, new_monomers):
  """ Set the monomers in the polymer to new_monomers. """
  self._monomers = new_monomers

def get_avg_age(self):
  totalage = 0
  for n in range(self.get_length()):
    totalage += self.get_monomers()[n].get_age()
  avgage= totalage/self.get_length()
  return avgage


def get_chirality(self):
  """ Return the chiralities of the monomers within the polymer.

  RETURN: tuple
  (number_lefts, number_rights)
  """
  #Get the monomer handednesses
  sequence = self.get_sequence()
  #Count the number of trues
  n_true = sum(sequence)
  #The number of falses is the total minus the number of trues
  #Return that.
  return (n_true, len(sequence) - n_true)

def get_signed_ee(self):
  lefts, rights = self.get_chirality()
  return (lefts - rights) / self.get_length()

def get_ee(self):
  lefts, rights = self.get_chirality()
  return abs((lefts - rights) / self.get_length())

#def get_leftrighthomochiralbonds(self):
  #"""
  #returns  # of left and right homochiral bonds in tuple (l,r)
  #"""
  #l=0

  #r=0

  #for n in range(self.get_length()):
    #if (self.easthomochiralbiascheck(n) == 3):
         # l += 1
    #elif (self.easthomochiralbiascheck(n) == 5):
        #  r += 1
  #tuple1 = (l , r)
  
  #return tuple1
  
def generate_old_log(self):
  """ Return a log tuple in the format
  (length, signed ee, total # homochiral bonds)
  """
  return (self.get_length(), 
          self.get_signed_ee())
          #sum(self.get_leftrighthomochiralbonds()))
  
def generate_log(self):
  """ Return a log tuple formatted simply as the list of boolean handednesses of the monomers contained
  in the polymer. Will be heavier than holding data about the polymer like in the other log format,
  but should allow access to more information and more kinds of information.
  """
  return tuple(self.get_sequence())

### Functionality methods

In [10]:
%%add_to Polymer

def append(self, other):
  """
  Adds a reactable (monomer or polymer) to the end of the Polymer.
  If the added reactable is a polymer, append() deletes it after taking its monomers.
  """
  #Check the type of thing to add
  if isinstance(other, Monomer):
    #Monomers can just be appended to the list
    self._monomers.append(other)
  elif isinstance(other, Polymer):
    #For polymers we need to extend rather than append, to avoid nested lists
    self._monomers.extend(other.get_monomers())
    #Then once the monomers are safely in their new polymer, the old one is deleted
    del other

def brkloc(self):

  #makes a list of all the indices of monomers in the polymer except 
  #for the last one
  indices = []
  for n in range(self.get_length() - 1):
    indices.append(n)
  #puts indices in random order
  random.shuffle(indices)
  #tests if each indices will break
  for index in indices:
    #gets the breakprobability for monomer at n index
    brkprob = self._monomers[n].get_eastbrkprob()
    rand = random.random()
    # checks if the random number is less than the brkprob then the index of
    #the monomer is returned from the method
    #indicating this polymer breaks at the east bond of this monomer
    if(brkprob > rand):
      return n
  return None

def removeright(self,location):
  """
  Removes all monomers to the right (east) of a given index
  """
  #creates a list of monomers in the polymer to the left, and including,
  #the monomer at the location(index) provided
  newList = self._monomers[0:location+1]

  #makes a list of the monoers to the rightof the monomer at the 
  #index provided
  removed = self._monomers[location+1:]

  #sets the polymers monomers to newList(the monomers to the left
  #of  and the index provided)
  self.set_monomers(newList)
  #print("after breaking ractable is:",self)
  #print("after breaking removed is:",removed)

  #returns the monomers that were removed, everything the right
  #of the index provided
  return removed

### Homochirality handling

In [11]:
%%add_to Polymer
def easthomochiralcheck(self,numbermonomer):
  """
  Takes the index of a monomer within the Polymer and returns whether its east bond is homochiral.
  """
  #First check that the index isn't out of bounds 
  if (0 > numbermonomer or numbermonomer >= self.get_length()-1):
    return False
  #get the handedness of this monomer and its east neighbor, and return whether or not they're equal (bool)
  return self._monomers[numbermonomer].get_handedness() == self._monomers[numbermonomer+1].get_handedness()

def easthomochiralbiascheck (self, numbermonomer):
  """ this method can be used in place of the east homochiral check method
  returns false if not homochiral, 3 if homochiral left, 5 if homochiral right"""

  #First check that the index isn't out of bounds or if not homochiral
  if (0 > numbermonomer or numbermonomer >= self.get_length()-1 or not self.easthomochiralcheck(numbermonomer)):
    return False
  
  #otherwise check if they are homochiral left or right
  elif self._monomers[numbermonomer].get_handedness():
      #means left homochiral
      return 3
  elif not self._monomers[numbermonomer].get_handedness():
      #means right homochiral
      return 5
  

def eastbondbreakprobability(self,numbermonomer):
  """
  Takes the index of a monomer within the Polymer
  Returns the probability that the monomer's east bond will break
  returns -3 if the monomer has no east bond
  """
  ##############################
  #Override to return an equal break chance for every bond in the polymer
  #Simply comment out this line to get the regular function
  #return (0.3)
  ##############################

  #First check if the monomer is the last one in the polymer
  if (numbermonomer >= self.get_length()-1):
    #If so, it doesn't have an east bond to break, so the probability is 0(return -3)
    return -3
    
  #now we initialize brk probability (brk)
  brk = (BASE_BOND_BREAK_PROBABILITY) * (LENGTH_FACTOR**(self.get_length()/N))
  #check if the east bond is homochiral
  if (self.easthomochiralcheck(numbermonomer)):
    #if so multiply it by homochiral break factor (shrinks probability)
    brk *= HOMOCHIRAL_BREAK_FACTOR
    #goes through method which checks and calculates benefit of all homochiral neighbors
    brk = self.checkforhomochiralneighbors(numbermonomer,brk,HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR)
  #end def: returns break probability of monomers east bond
  return brk

def biaseastbondbreakprobability(self,numbermonomer):
  """
  ***left-right sensitivity****
  Takes the index of a monomer within the Polymer
  Returns the probability that the monomer's east bond will break
  returns -3 if the monomer has no east bond
  """
  ##############################
  #Override to return an equal break chance for every bond in the polymer
  #Simply comment out this line to get the regular function
  #return (0.3)
  ##############################
  
  #First check if the monomer is the last one in the polymer
  if (numbermonomer >= self.get_length()-1):
    #If so, it doesn't have an east bond to break, so the probability is 0 (returns -3)
    return -3
  
  #initialize the brk probability
  brk = (BASE_BOND_BREAK_PROBABILITY) * (LENGTH_FACTOR**(self.get_length()/N))

  #check if the east bond is homochiral left
  if (self.easthomochiralbiascheck(numbermonomer) == 3):
    brk *= HOMOCHIRAL_BREAK_FACTOR_LEFT

    #run through function that recalculates brk based on benfits of homochiral neighbors
    brk = self.checkforhomochiralneighbors(numbermonomer, brk, HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT)

  #otherwise checks if bond is homochiral right
  elif (self.easthomochiralbiascheck(numbermonomer) == 5):
    brk *= HOMOCHIRAL_BREAK_FACTOR_RIGHT

    #run through function that recalculates brk based on benfits of homochiral neighbors
    brk = self.checkforhomochiralneighbors(numbermonomer,brk, HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT)
   
  return brk

def checkforhomochiralneighbors(self, numbermonomer, brk, neighborfactor):

  """
  helps all the versions of the eastbond break calculating functions
  takes in a brk probability a number monomer and the desired neighbor improvement factor
  and calculates how the neighbors a bond will increase the strength of the bond
  it then returns the new brk prob to the eastbond brk function
  """
  brk = brk
  j = numbermonomer + 1
  #going right to check for homochiral neighbors
  while(self.easthomochiralcheck(j)):
    #calculates decrease to brk prob for a neighbor of that distance for the bond
    brk *= 1 - (neighborfactor**abs(j - numbermonomer))
    j += 1

  #going left to check for homochiral neighbors
  j=numbermonomer-1
  while(self.easthomochiralcheck(j)):
    #calculates decrease to brk prob for a neighbor of that distance for the bond
    brk *= 1-(neighborfactor**abs(numbermonomer - j))
    j -= 1

  return brk


def poissonbreakprobability(self,numbermonomer):
  """
  Takes the index of a monomer within the Polymer
  Returns the probability that the monomer's east bond will break
  based on Poisson distribution
  returns -3 if the monomer has no east bond
  """

  #First check if the monomer is the last one in the polymer
  if (numbermonomer >= self.get_length()-1):
    #If so, it doesn't have an east bond to break, so the probability is 0(return -3)
    return -3

  #With that out of the way, we initialize brk to the base probability of breaking a bond
  brk = (BASE_BOND_BREAK_PROBABILITY) * (LENGTH_FACTOR**(self.get_length()/N))

  #keeps track of the number of the total consecutive homochiral bonds
  homochiralcount=0 

  #check if the east bond is homochiral
  if (self.easthomochiralcheck(numbermonomer)):
    #if so homochiral count increases by 1
    homochiralcount +=1
    #brk gets benefited by the poisson distribution of homochiral count 
    brk *= 1-(1.3*(self.poissonequation(homochiralcount)))

    #set j to monomer to right
    j = numbermonomer+1

    #going right to check for neighboring homochirality
    while(self.easthomochiralcheck(j)):
      #if homochiral increases count
      homochiralcount += 1
      #recalculates brk prob
      brk *= 1-(1.3*(self.poissonequation(homochiralcount)))

      #check next monomer
      j += 1

    #going left to check for neighboring homochirality
    j=numbermonomer-1
    while(self.easthomochiralcheck(j)):
      homochiralcount += 1
      brk *= 1-(1.3*(self.poissonequation(homochiralcount)))
      j -= 1

  return brk

def poissonequation(self,k):
  #checks if value has already been calulated
  if k not in poisson_dict:
    #otherwise calculates the values
    p = np.exp(-LAMBDA)*((LAMBDA**k)/(np.math.factorial(k)))
    #adds to dictionary
    poisson_dict[k] = p
  return poisson_dict[k]



def reset_break_probability(self):
    
  """ 
  resets the break probabilities of the monomers in a 
  sequence. 1)checks lookup table to see if similar polymer values
  have been caluclated, otherwise calculates values and adds
  polymer to lookup table 2)sets values to monomers.
 
  """
  #store self as list of booleans to be added to lookup table
  polyseq = self.get_monomers().copy()
  for n in range(self.get_length()):
    polyseq[n] = polyseq[n].get_handedness()


  # All stored sequences start with True. If the one being requested doesn't, invert so that it does
  #comment out if right/left sensitivity is included
  if polyseq[0] == True:
    polyseq = [not mono for mono in polyseq]

  #make polyseq a tuple able to be added as an index of a dictionary  
  polyseq = tuple(polyseq)

  # If the sequence is already in the table (dict), retrieves its previously calculated break probabilities
  if polyseq in break_prob_lookup_table:
    the_probs = break_prob_lookup_table[polyseq]
    
  else: # Otherwise calculate and add to the lookup table
    # Calculate the break probabilities
    the_probs = self.calculatebrkprob()
    #and add it to the lookup table
    break_prob_lookup_table[polyseq] = the_probs

  #set monomers eastbond brk probs to correlated values stored in the_probs
  for n in range (self.get_length()):
    self._monomers[n].set_eastbrkprob(the_probs[n])

  #end def: polymer break probabilities recalculated and added
  #to lookup table (if not already there)

def calculatebrkprob(self):
    
  """
  generates and returns a list of break probabilities for the monomers at each index of
  a polymer. This method can be altered to calculate normally
  with left/right sensitivity or with a poisson distribution
 
  """

  #initialize brk_probs as an empty list
  brk_probs = []
  
  #checks if polymer is of length one -meaning it is lone monomer
  if self.get_length() == 1 :
    #if so adds -1 to list(brk prob value assigned to individual monomers)
    brk_probs.append(-1)
    
  else:
    #otherwise goes through all the indices of the polymer,calculates its brk probability, and appends it to brk_probs 
    for n in range(self.get_length()):

      #############################################################
      #LEFT RIGHT SENSITIVITY CHOICE/poisson equation options     #
      #pick one-comment out the rest.                             #
      #############################################################

      #this line if uncommented will have left/right sensitivity
      #brk_probs.append(self.biaseastbondbreakprobability(n))

      #this line if uncommented will calculate probabilities normally
      #brk_probs.append(self.eastbondbreakprobability(n))

      #this line if uncommented will calculate brk probabilities with poisson distribution
      brk_probs.append(self.poissonbreakprobability(n))

  #returns in order list of brk probs for the east bond of all the monomers in a polymer
  return brk_probs


## The Reactables class

This is the Reactables class aka the stylish little bag we keep our contestants in. As all good dating shows, we define our members by the relationship they are in. So a single monomer will be addd to the bag alone, but a monomer in a polymer will not be. Rather each polymer will be recorded once. Its like if you have charlie, ben, and jack. Jack and Ben are in a relationship so they are jointly called Jen. If these three were in the reactable bag they would be recognized as two things, charlie and Jen(Ben+JAck). The reactable class is the heart and center of our game- I mean dating- show. Its here we ask all the members just the right questions to get them falling in love and breaking hearts, and then we are curteous enough to do all the splitting and merging for them. 

### Constructor

In [12]:
class Reactables:

  def __init__(self, reactables = []):
    """
    Constructor class for Reactables object
    Accepts list of reactables or defaults to empty list
    Reactables functions as a bin for reactable objects monomer and polymer
    Methods consist mostly of list handling
    """
    self._reactables = reactables
    #The reactables bag also handles remembering its history
    self._history = []
    #self._leftrighthistory = []
    #list of tuples of avg age and signed ee for each reactable in the list
    self._agehistory = []


  def __str__(self):
    """
    str method for Reactables class
    """
    return str(self.get_reactables())

### Getters and Setters

In [13]:
%%add_to Reactables

#################
#GETTERS/SETTERS#
#################

def get_reactables(self):
  """
  Getter method for reactables in Reactables
  Returns a list of the objects currently in the Reactables bag
  """
  return self._reactables

def get_history(self):
  """ Return the history of the reactables bag.
  """
  return self._history

#def get_leftrighthistory(self):
  #return self._leftrighthistory

def get_agehistory(self):
  return self._agehistory
  
def get_count(self):
  """
  Getter method for Reactables class
  Returns the number of objects in the Reactables bag
  """
  return len(self.get_reactables()) 


def set_reactables(self, new_list):
  """
  Setter method for the Reactables class
  Sets the reactables bag to a new list
  Used mostly as a helper function in bonding
  """
  self._reactables = new_list

def set_agehistory(self, newagehistory):
  self._agehistory = newagehistory


def get_bond_chance(self, reactable1, reactable2):
  """
  Returns the probability that two given reactables in the reactables bag will bond
  """
  return BOND_PROB

def get_free_monomers(self):
  return [r for r in self.get_reactables() if isinstance(r, Monomer)]

def get_polymers(self):
  return [r for r in self.get_reactables() if isinstance(r, Polymer)]

def get_bound_monomers(self):
  bound = []
  for p in self.get_polymers():
    bound.extend(p.get_monomers())
  return bound

#def record_leftrighthomochiral(self):
 # """returns the number of left and right homochiral bonds, (left,right)"""
 # l=0
  #r=0
  #for polymer in self.get_polymers() :
   # tuple1 = polymer.get_leftrighthomochiralbonds()
    #l += tuple1[0]
    #r += tuple1[1]
  #tuple2 = (l , r)
  #self._leftrighthistory.append(tuple2)
  #return tuple2

### Functionality methods

#### Top level functionality

In [14]:
%%add_to Reactables

def refill(self, pool_size):
  #Do we have less than the required number of things in the pool?
  dearth = pool_size - self.get_count()
  #If so,
  if dearth > 0:
    new = []
    #Make as many monomers as we need to get back up to the riquired amount
    for n in range(dearth):
      new.append(Monomer())
    #And add them to the reactables bag
    self.add(new)

def randomize_reactables(self):
  """
  Randomizes the order of the reactables list
  """
  #Get the current reactables
  reactables = self.get_reactables()
  #Shuffle them
  random.shuffle(reactables)
  #And reset the reactables list to the new sorted list
  self.set_reactables(reactables)

def do_the_thing(self):
  """Handle a single iteration of the reactables.

  """
  #We keep track of bonding by storing the most recent reactable to choose to bond as the 'bachelor.'
  bachelor = None
  #We need a copy of the reactables because indices will change as soon as we start doing stuff
  reactables = copy.copy(self.get_reactables())
  #Iterate through each reactable
  for item in reactables:
    #Roll a random number in (0,1) to compare against our probabilities
    roll = random.random()
    #If our reactable is a monomer...
    if isinstance(item, Monomer):
      #We get our breaking and bonding chances (parameters now, may become functions later?)
      brk = POOF_CHANCE
      bond = BOND_PROB
      #If we roll to break the monomer
      if 0 < roll <= brk:
        #Just delete it
        del item
      #If we roll to bond the monomer
      elif brk < roll <= brk+bond:
        #Check if there's a reactable waiting to bond
        if bachelor == None:
          #If not, make this monomer the bachelor. Someone will come along to bond later.
          bachelor = item
        else:
          #If there is someone waiting, bond them together and reset the bachelor chair to empty.
          self.bond_pair(bachelor, item)
          bachelor = None
      else:
        continue
    #If the reactable is a polymer instead...
    elif isinstance(item, Polymer):
      #Choose whether the polymer will break or bond. It's 50-50 right now.
      if roll >= 0.5:
        #This is the same bonding logic as for the monomer. It's not in a helper function because it needs to be able to access bachelor.
        if bachelor == None:
          bachelor = item
        else:
          self.bond_pair(bachelor, item)
          bachelor = None
      #Here's what happens if the polymer is chosen to check breaking
      else:
        #It finds its break location
        break_spot = item.brkloc()
        #Which might turn out to be nowhere.
        if break_spot != None:
          #If somewhere does break, call break_polymer to handle the breaking
          self.break_polymer(item,break_spot)
    else:
      #You never know what might end up in your reactables bag
      raise ValueError("This thing is neither a monomer or a polymer. What?!")

def iterate(self,size):
  """
  Handles full sequence of iteration
  """
  self.randomize_reactables()
  self.do_the_thing()
  self.log()
  #self.record_leftrighthomochiral()
  self.refill(size)

#### Functionality helpers

In [15]:
%%add_to Reactables


def add(self, new_reactables):
  """
  Adds a list of reactables to the reactables bag
  Built on extend(); argument must be iterable
  """
  #print("list of new Reactbles", new_reactables)
  #print("list of get Reactables:",self.get_reactables())
  self.get_reactables().extend(new_reactables)
  #print("reactables list is now",self.get_reactables())


def subtract(self, removables):
  """
  Removes a list of reactables from the reactables bag
  Built on list comprehension; argument must be iterable
  """
  self.set_reactables([reactable for reactable in self.get_reactables() if not (reactable in removables)])


def bond_pair(self, west, east):
  """
  Accepts two reactables from the bag
  Bonds them together, handling deletion of any emptied polymer
  """
  #Take your two reactables and check if the west one is a monomer or a polymer 
  if isinstance(west, Polymer):
    #If it's a polymer, all you've got to do is append the east item and then take it out of the bag
    west.append(east)
    self.subtract([east])
    #recalculate the breakprobabilities of the bonds in polymer
    west.reset_break_probability()
  if isinstance(west, Monomer):
    #If west friend is a Monomer, then make a new polymer containing west
    newpoly = Polymer([west])
    #Add the east things to it
    newpoly.append(east)
    #Remove both the west monomer and the east thing
    self.subtract([east])
    self.subtract([west])
    #And put the new polymer into the reactables bag
    self.add([newpoly])
    #recalculates break probability
    newpoly.reset_break_probability()


def break_polymer(self, polymer, brk_location):
  #breaks polymer at given location and creates a new polymer of the 
  #monomers removed
  #when polymer is made-breakprobabilities are calculated
  newPolymer = Polymer(polymer.removeright(brk_location))
  #if(newPolymer._monomers[0].get_handedness() and polymer._monomers[-1].get_handedness()):
    #print ("left breaking when literally impossible????")
  polys = [newPolymer, polymer]
  #resets break probabilities in polymer
  polymer.reset_break_probability()
  #goes throught the two new polymers
  for poly in polys:
    if poly.get_length()==0:
      print("whyyyyyyyyyyy is this happening????")
    #checks if they are of length 1
    if poly.get_length() <= 1:
      #adds them to reactable as a single monomer
      self.add(poly.get_monomers())
      #subtracts polymer from reactable list
      self.subtract([poly])
      del poly
    #checks if the polymer is not in the reactables list
    elif (poly not in self.get_reactables()):
      #add polymer to reactable list
      self.add([poly])
    



### Current state data methods

In [16]:
%%add_to Reactables

def get_overall_chirality(self):
  """ Return the total quantity of left- and right-handed monomers in the reactable
    pool, including monomers within polymers.

  RETURN tuple
  (Left-handed_quantity, Right-handed_quantity)
  """
  left_count = 0
  right_count = 0
  for reactable in self.get_reactables():
    if isinstance(reactable, Monomer):
      if reactable.get_handedness():
        left_count += 1
      else:
        right_count += 1
    elif isinstance(reactable, Polymer):
      l, r = reactable.get_chirality()
      left_count += l
      right_count += r
    else:
      print(f"{reactable}, which is a {type(reactable)}")
      raise ValueError
  return (left_count, right_count)

def get_polymer_chirality(self):
  """ Return the total quantity of left- and right-handed monomers in the polymers
  within the reactable pool.

  RETURN tuple
  (Left-handed_quantity, Right-handed_quantity)
  """
  left_count = 0
  right_count = 0
  for reactable in self.get_reactables():
    if isinstance(reactable, Polymer):
      l, r = reactable.get_chirality()
      left_count += l
      right_count += r
  return (left_count, right_count)

def get_free_proportion(self):
  """Return the proportion of free monomers to total population"""
  return len(self.get_free_monomers())/self.unit_count()

def unit_count(self):
  """return the total number of monomers in the bag, both free and bound"""
  bound = 0
  for p in self.get_polymers():
    bound += p.get_length()
  return len(self.get_free_monomers()) + bound

def get_polymer_ee(self):
  ee = []
  for polymer in self.get_polymers():
    ee.append(polymer.get_signed_ee())
  return ee

def avg_ee_by_length(self):
  """ Return a list containing the enantiomeric excess of all reactables sorted by size.
  That is:
  [(ee of monomers),(ee of 2-mers),(ee of 3-mers) etc]
  """
  output = []
  for n in range(1,self.max_length()+1):
    group = self.get_reactables_by_length(n)
    ees = []
    # print(f"For length {n} I got {len(group)} reactables.")
    if len(group) > 0:
      # eestr = ""
      for r in group:
        ees.append(r.get_ee())
        # eestr += f", {r.get_ee()}"
      output.append(np.mean(ees))
      # print(f"The ee's of this length were {eestr}.\nThe average ee was {np.mean(ees)}.")
    else:
      output.append(0)
  return output

def get_reactables_by_length(self,length):
  if length <= 1:
    #return the monomers
    return self.get_free_monomers()
  else:
    return [p for p in self.get_polymers() if p.get_length() == length]

def max_length(self):
  """ Return the length of the longest polymer in the reactables bag.
  """
  length = 0
  for p in self.get_polymers():
    if p.get_length() > length:
      length = p.get_length()
  return length

def logagehistory (self):
  log = []
  polymers = self.get_polymers()
  monomers = self.get_free_monomers()
  for molecule in polymers :
    representation = (molecule.get_avg_age(), molecule.get_signed_ee())
    log.append(representation)
  for molecule in monomers :
    respresentation = (molecule.get_age(), molecule.get_signed_ee())
    log.append(representation)
  return log
  



### History handling methods

In [17]:
%%add_to Reactables

# def old_log(self):
#   """ Translate current reactables list into a lighter-weight list of tuples 
#     describing the bag state, then logs that list into self._history.
#     Log format: (length,signed_ee)
#     e.g.  True-monomer would be (1,1)
#           [False, True, False, False]-polymer would be (4,-0.5)
#     """
#   log = []
#   for r in self.get_reactables():
#     log.append(r.generate_old_log())
#   (self._history).append(log)

def log(self):
  """ Translate current reactables into a loggable list of tuples.
  This log keeps the full sequences of the polymers without digesting the data, 
  so it should be more flexible as we more forward with pulling new information
  out of this simulation. 
  There is one major piece of information lost, however, which is the identities
  of the individual polymers and monomers. """
  log = []
  for r in self.get_reactables():
    log.append(r.generate_log())
  (self._history).append(log)
  self._agehistory.append(self.logagehistory())
  




# def get_iter_polymers_by_length(self, single_iteration_log):
#   # So we have something like this: [(t,f,f),(t),(f,f),(f),(t,t,t,f)]
#   polymers_by_length = []
#   lengths = [len(log) for log in single_iteration_log]
#   biggest = max(lengths)
#   for n in range(1, biggest+1):
#     polys = [log for log in single_iteration_log if len(log) == n]
#     if len(polys) > 0:
#       polymers_by_length.append(polys)
#     else:
#       polymers_by_length.append([])
#   return polymers_by_length

# def get_iter_ees_by_length(self,single_iter_log):
#   polymers_by_length = self.get_iter_polymers_by_length(single_iter_log)
#   for length_category in polymers_by_length:
#     for polymer in length_category:
#       ee = hist_get_polymer_ee(polymer)






#### Low level info functions

In [18]:
%%add_to Reactables

def hist_get_polymer_homochirality_of_bonds(self,polylog):
  """ Return a boolean list of bonds within a given logged polymer, True if the bond is
      homochiral. """
  return [i[0]==i[1] for i in list(more_itertools.pairwise(polylog))]

def hist_get_polymer_ee(self,polylog):
  """ Return the (signed) enantiomeric excess of a logged polymer. """
  length = len(polylog)
  n_True = sum(polylog)
  n_False = length - n_True
  return (n_True - n_False) / length

def hist_get_polymers(self,iteration):
  """ Return a list of the polymer logs in an iteration. This is just the iteration
  but without the monomers. """
  return [r for r in iteration if len(r) > 1]

def hist_count_longest_homochiral_chain(self,polylog):
  """ Return the length of the longest homochiral chain given the log of a polymer. """
  previous = None
  count = 1
  longest = 1
  for monomer in polylog:
    if monomer == previous:
      count += 1
    else:
      longest = max(count, longest)
      count = 1
    previous = monomer
  longest = max(count, longest)
  return longest

def hist_get_polymer_chain_lengths(self,polylog):
  """ Return the lengths of the homochiral chains in a given polymer log. """
  count = 1
  lengths = [] 
  for n in range(1,len(polylog)):
    if polylog[n] == polylog[n-1]:
      count += 1
    else:
      lengths.append(count)
      count = 1
  lengths.append(count)
  return lengths

def hist_get_iteration_chain_lengths(self,iteration):
  """ Return the lengths of all the homochiral chains in polymers in a given iteration. """
  polymers = self.hist_get_polymers(iteration)
  chain_lengths = []
  for polymer in polymers:
    chain_lengths.extend(self.hist_get_polymer_chain_lengths(polymer))
  return chain_lengths

def hist_get_leftright_homochiral_count(self,iteration):
  """
  returns a tuple (l,r)of the total number of left homochiral bonds and right homochiral bonds in an iteration 
  """
  
  l=0
  r=0
  for reactable in iteration:
      for monomer in range(len(reactable)-1):
        if (reactable[monomer]== reactable[monomer+1]):
          if (reactable[monomer]):
            l += 1
          elif (not reactable[monomer]):
            r += 1
  leftrightcount=(l,r)
  return leftrightcount

### Plotting methods

In [19]:
%%add_to Reactables

def visualize_parameters(self,stamp=""):
  fig,ax = plt.subplots()

  parameters = ['POOF_CHANCE','BOND_PROB','BASE_BOND_BREAK_PROBABILITY',
                'HOMOCHIRAL_BREAK_FACTOR','HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR',
                'LENGTH_FACTOR','HOMOCHIRAL_BREAK_FACTOR_LEFT',
                'HOMOCHIRAL_BREAK_FACTOR_RIGHT',
                'HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT',
                'HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT']
  values = [POOF_CHANCE,BOND_PROB,BASE_BOND_BREAK_PROBABILITY,
            HOMOCHIRAL_BREAK_FACTOR,HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR,
            LENGTH_FACTOR,HOMOCHIRAL_BREAK_FACTOR_LEFT,
            HOMOCHIRAL_BREAK_FACTOR_RIGHT,
            HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_LEFT,
            HOMOCHIRAL_NEIGHBOR_IMPROV_FACTOR_RIGHT]

  ypos = np.arange(len(parameters))

  ax.barh(ypos, values, align='center')
  ax.set_yticks(ypos)
  ax.set_yticklabels(parameters)
  ax.invert_yaxis()  # labels read top-to-bottom
  ax.set_xlabel('Value')
  ax.set_title(f"{stamp}Parameter values")



def plot_signed_ee_spread(self,cmap='bone',stamp=""):
  #First make the polymerspread thingy
  poly_ee_hist = []
  for iteration in self.get_history():
    polys = [log for log in iteration if not isinstance(log,bool)]
    polydata = [log for log in polys if len(log) > 1] # All the polymers (not monomers) in the iteration
    poly_ees = [self.hist_get_polymer_ee(poly) for poly in polydata] # the EE of each polymer in the iteration
    poly_ee_hist.append(poly_ees) # Put the EE list in our plottable structure
  #Now we can make the plot
  fig,ax = plt.subplots()
  ys = []
  xs = []
  for n in range(len(poly_ee_hist)):
    y = poly_ee_hist[n]
    x = [n] * len(y)
    ys.extend(y)
    xs.extend(x)
  ax.hexbin(xs,ys,cmap=cmap)
  ax.set_title(f"{stamp}polymer spread")
  ax.set_xlabel("iteraions")
  ax.set_ylabel("ee")

def plot_ee_heatmap(self,cmap='bone',stamp=""):
  #Need to figure out how to get ees_by_length from history
  ees_by_length = []
  for iteration in self.get_history():
    ee_data = []
    #get the polymers
    polys = [log for log in iteration if not isinstance(log,bool)]
    #get the length of the longest polymer
    lengths = [len(log) for log in polys]
    try:
      biggest = max(lengths)
    except:
      biggest = 0
    for n in range(1,biggest+1):
      ees = [abs(self.hist_get_polymer_ee(log)) for log in polys if len(log) == n]
      if len(ees) > 0:
        ee_data.append(np.mean(ees))
      else:
        ee_data.append(0)
    ees_by_length.append(ee_data)
  #Need to make a new ee array with rectangular dimensions
  ee = numpy_fillna(ees_by_length)
  fig,ax = plt.subplots(dpi=150)
  im = ax.imshow(ee,cmap=cmap)
  ax.set_title(f"{stamp}ee of polymers by length through iterations")
  ax.set_xlabel("polymer lengths")
  ax.set_ylabel("iterations")
  fig.tight_layout()
  plt.show()

def plot_leftrighthomochirality(self):
  plt.figure()
  L=[]
  R=[]
  x=[]
  i=1

  for iteration in self.get_history():
    leftrightcount=self.hist_get_leftright_homochiral_count(iteration)
    L.append(leftrightcount[0])
    R.append(leftrightcount[1])
    #for reactable in iteration:
     # for monomer in range(len(reactable)-1):
       # if (reactable[monomer]== reactable[monomer+1]):
         # if (reactable[monomer]):
          #  l += 1
         # elif (not reactable[monomer]):
           # r += 1
    #L.append(l)
    #R.append(r)
    x.append(i)
    i+=1


  #for iteration in self.get_leftrighthistory():
    #l.append(iteration[0])
    #r.append(iteration[1])
    #x.append(i)
    #i+= 1
  plt.plot(x,L,label="left homochiral")
  plt.plot(x,R,label="right homochiral")
  plt.ylabel("Number of bonds")
  plt.xlabel("Iterations")
  plt.title("Left right homochiral bonds when LEFT="+str(HOMOCHIRAL_BREAK_FACTOR_LEFT) +  "and right=" + str(HOMOCHIRAL_BREAK_FACTOR_RIGHT))
  plt.legend()
  plt.show()

def plot_homochiral_chain_lengths(self):
  """ Plot a chart of the mean length of homochiral chains present at each iteration.
  The plot generated will attempt to summarize the mean lengths of homochiral
  chains within polymers across iterations. It will also attempt to visualize the 
  varience with continuous lines plotting the standard deviation around that mean.
  
  The x axis of the resulting plot is iterations, and the y axis is length of 
  the homochiral chains. Both of these are discrete categories, but the plot will
  treat iterations as continuous to improve readability for large n.
  """
  # Get the history and read it 
  history = self.get_history()
  means = []
  stdevs= []
  maxes = []
  for iteration in history:
    # Get the chain lengths in that iteration
    iter_data = self.hist_get_iteration_chain_lengths(iteration)
    means.append(np.mean(iter_data))
    stdevs.append(np.std(iter_data))
    maxes.append(max(iter_data))
  # Plot those data
  fig,ax = plt.subplots()
  ax.fill_between(np.arange(0,len(means)), [m + s for m, s in zip(means, stdevs)],[m - s for m, s in zip(means, stdevs)], alpha=0.2, label = "Means +- one standard deviation")
  ax.plot(means, 'k-', label = "Mean homochiral chain length")
  ax.plot(maxes, 'b.', label = "Max homochiral chain length")
  ax.set_xlabel("Iteration")
  ax.set_ylabel("Homochiral chain length")
  ax.legend()

def plot_homochirality_vs_length(self):
  """ Take all the polymers that ever existed in history and plot them on a
  scatter plot of length vs homochirality. Homochirality is the percent of their
  bonds that are homochiral. """
  #Get the history
  history = self.get_history()
  #Get the polymers
  x = []
  y = []
  for i in history:
    for p in self.hist_get_polymers(i):
      homochirality = sum(self.hist_get_polymer_homochirality_of_bonds(p))/len(p)
      x.append(len(p))
      y.append(homochirality)
  fig,ax = plt.subplots()
  ax.scatter(x,y)
  ax.set_xlabel("Polymer lengths")
  ax.set_ylabel("Homochirality")

def plot_age_vs_homochirality(self):
  plt.figure()
  for iteration in self.get_agehistory():
    for reactable in iteration:
      plt.plot(reactable[0],reactable[1])
  plt.xlabel("avg age")
  plt.ylabel("Signed_ee")
  plt.show()



        



    


  


## Test program

In [20]:
POOL_SIZE = 10
ITERATIONS = 100


a = Reactables(make_pool(POOL_SIZE))

print("Running.............|\n", end="")

monomercounts = []

for n in range(ITERATIONS):
  a.iterate(POOL_SIZE)
  monomercounts.append(len(a.get_free_monomers())+len(a.get_bound_monomers()))
  if n % (ITERATIONS / 10) == 0:
    print("##",end="")

print("\nDone.")

plots = ["parameters","polymer ee spread","ee heatmap","leftright chirality",
         "homochiral chain lengths", "homochirality vs length","Homochirality vs age"]
tb = widgets.TabBar(plots)

with tb.output_to("parameters"):
  a.visualize_parameters()

with tb.output_to("polymer ee spread"):
  a.plot_signed_ee_spread()

with tb.output_to("ee heatmap"):
  a.plot_ee_heatmap()

with tb.output_to("leftright chirality"):
  a.plot_leftrighthomochirality()

with tb.output_to("homochiral chain lengths"):
  a.plot_homochiral_chain_lengths()

with tb.output_to("homochirality vs length"):
  a.plot_homochirality_vs_length()

with tb.output_to("Homochirality vs age"):
  a.plot_age_vs_homochirality()





Running.............|


AttributeError: ignored

# a spot for checking code things

In [ ]:
N=30
testlengthfactor =0.6
def lengthfactor(length):

 brk = (BASE_BOND_BREAK_PROBABILITY) * (testlengthfactor**(length/N))
 return brk

plt.figure()
x=[0,2,3,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,22,23,24,25,26,27,28,29,30]
y=[]
for int in x:
  y.append(lengthfactor(int))
plt.plot(x,y)
plt.show()


# Observations

Under the current refill rules, the upper bound for the total number of monomers (both free and within polymers) in the world is $1/2(P)(I)$ for $P$ = pool size and $I$ = number of iterations.

The upper bound on possible polymer size is 

$
\begin{equation}
  bound=\left\{
  \begin{array}{@{}ll@{}}
    2^n, & n \leq\ T \\
    2^T+(n-T)(2^{T-1}), & n>T
  \end{array}\right.
\end{equation} 
$

for $T=log_2(P)$.

For $P < 100$ or so, these bounds are substantially lower than $2^n$, but for large $P$ it matters less.

Also $2^T+(n-T)(2^{T-1}) = P(1+1/2(n-log_2(P))$ I think.